In [ ]:
import aiohttp
import asyncio
import pandas as pd
import time
import random
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
import nest_asyncio

nest_asyncio.apply()

topics = {
    "Artificial Intelligence": "cs.AI",
    "Hardware Architecture": "cs.AR",
    "Computation and Language": "cs.CL",
    "Computer Vision and Pattern Recognition": "cs.CV",
    "Databases": "cs.DB",
    "Data Structures and Algorithms": "cs.DS",
    "Formal Languages and Automata Theory": "cs.FL",
    "Computer Science and Game Theory": "cs.GT",
    "Human-Computer Interaction": "cs.HC",
    "Information Retrieval": "cs.IR",
    "Machine Learning": "cs.LG",
    "Sound": "cs.SD",
    "Robotics": "cs.RO",
    "Software Engineering": "cs.SE",
    "Cryptography and Security": "cs.CR"
}
VALID_CATEGORIES = set(topics.values())

data = []
CHECKPOINT_INTERVAL = 5000

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

async def fetch_page(session, url):
    try:
        async with session.get(url, headers=headers, timeout=5) as response:
            response.raise_for_status()
            return await response.text()
    except Exception:
        return None

def parse_total_entries(html):
    soup = BeautifulSoup(html, "html.parser", from_encoding="utf-8")
    paging_div = soup.find("div", class_="paging")
    if paging_div:
        match = re.search(r"Total of (\d+) entries", paging_div.text)
        return int(match.group(1)) if match else 0
    return 0

async def scrape_abstract(session, link, title):
    html = await fetch_page(session, link)
    if not html:
        return None

    soup = BeautifulSoup(html, "html.parser", from_encoding="utf-8")
    subjects_elem = soup.find("td", class_="tablecell subjects")
    if not subjects_elem:
        return None

    categories = re.findall(r'\(([\w\.]+)\)', subjects_elem.text)
    matching_categories = set(categories) & VALID_CATEGORIES
    if not matching_categories:
        return None

    matched_category = next(iter(matching_categories))
    category_name = next((name for name, code in topics.items() if code == matched_category), "Unknown")

    abstract_elem = soup.find("blockquote", class_="abstract")
    abstract = abstract_elem.text.replace("Abstract:", "").strip() if abstract_elem else "Not found"

    return {
        "Title": title,
        "Link": link,
        "Category": category_name,
        "Abstract": abstract
    }

async def scrape_page(session, page_url, year):
    html = await fetch_page(session, page_url)
    if not html:
        return []

    soup = BeautifulSoup(html, "html.parser", from_encoding="utf-8")
    article_list = soup.find("dl", id="articles")
    if not article_list:
        return []

    dt_elements = article_list.find_all("dt")
    dd_elements = article_list.find_all("dd")

    tasks = []
    for dt, dd in zip(dt_elements, dd_elements):
        link_elem = dt.find("a", title="Abstract")
        if not link_elem:
            continue
        link = "https://arxiv.org" + link_elem["href"]
        title_elem = dd.find("div", class_="list-title")
        if not title_elem:
            continue
        title = title_elem.text.replace("Title:", "").strip()
        tasks.append(scrape_abstract(session, link, title))

    if not tasks:
        return []

    page_data = []
    for result in await asyncio.gather(*tasks, return_exceptions=True):
        if isinstance(result, dict):
            page_data.append(result)

    return page_data

async def scrape_year(year):
    base_url = f"https://arxiv.org/list/cs/{year}"
    initial_url = f"{base_url}?skip=0&show=50"

    async with aiohttp.ClientSession() as session:
        html = await fetch_page(session, initial_url)
        if not html:
            print(f"Failed to load initial page for year {year}")
            return

        total_entries = parse_total_entries(html)
        if total_entries == 0:
            print(f"Could not determine total entries for year {year}")
            return

        print(f"Scraping year {year} with {total_entries} total entries")
        entries_per_page = 50
        total_pages = (total_entries + entries_per_page - 1) // entries_per_page
        page_urls = [f"{base_url}?skip={page * entries_per_page}&show={entries_per_page}" 
                     for page in range(total_pages)]

        with tqdm(total=total_pages, desc=f"Year {year} Progress") as pbar:
            for i in range(0, len(page_urls), 10):  
                batch_urls = page_urls[i:i + 10]
                tasks = [scrape_page(session, url, year) for url in batch_urls]
                for page_data in await asyncio.gather(*tasks, return_exceptions=True):
                    if isinstance(page_data, list):
                        data.extend(page_data)
                        if len(data) // CHECKPOINT_INTERVAL > (len(data) - len(page_data)) // CHECKPOINT_INTERVAL:
                            save_checkpoint()
                    pbar.update(1)
                await asyncio.sleep(random.uniform(0.5, 1.5))

def save_checkpoint():
    df = pd.DataFrame(data)
    df.to_csv("/kaggle/working/arxiv_cs_2017_2025.csv", index=False, encoding="utf-8")
    print(f"Checkpoint: {len(data)} articles saved")

if __name__ == "__main__":
    for year in range(2017, 2026):
        asyncio.run(scrape_year(year))
        time.sleep(random.uniform(2, 5))

    if data:
        df = pd.DataFrame(data)
        df.to_csv("/kaggle/working/arxiv_cs_2017_2025.csv", index=False, encoding="utf-8")
        print(f"Final save: {len(data)} articles saved to /kaggle/working/arxiv_cs_2017_2025.csv")
    else:
        print("No data scraped")

/usr/local/lib/python3.10/dist-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Scraping year 2017 with 32728 total entries


Year 2017 Progress:  52%|█████▏    | 339/655 [04:48<03:28,  1.52it/s]

Checkpoint: 5005 articles saved


Year 2017 Progress: 100%|██████████| 655/655 [09:31<00:00,  1.15it/s]


...

Checkpoint: 225013 articles saved


Year 2025 Progress:  72%|███████▏  | 472/656 [09:29<04:13,  1.38s/it]

Checkpoint: 230026 articles saved


Year 2025 Progress: 100%|██████████| 656/656 [12:53<00:00,  1.18s/it]


Final save: 234170 articles saved to /kaggle/working/arxiv_cs_2017_2025.csv
